# Assignment10 : Binary Classifiers with various parameters to classify MNIST Dataset
# Name : Joonoh-Oh
# Student-ID : 20146136
# Github : https://github.com/jooh95/assignment10

# Set the training data file and test data file.

In [1]:
import numpy as np

file_data_train = "mnist_train.csv"
file_data_test  = "mnist_test.csv"

h_data_train    = open(file_data_train, "r")
h_data_test     = open(file_data_test, "r")

data_train      = h_data_train.readlines()
data_test       = h_data_test.readlines()

h_data_train.close()
h_data_test.close()

size_row    = 28    # height of the image
size_col    = 28    # width of the image

num_train   = len(data_train)   # number of training images
num_test    = len(data_test)    # number of testing images

# Define the various size of parameter p.

In [2]:
p = np.array([5, 25, 125, 625])

# Define the feature function with a random vector in a normal distribution.

In [3]:
def f(x, p_r, r):
    
    y = np.empty(p_r, dtype=float)

    for i in range(0, p_r):
        y[i] = np.dot(r[i].T, x)
    
    return y

# Define an activation function which is the argmax function.

In [4]:
def argmax(x):
    
    max_x = -9999
    
    idx = 0
    
    for i in range(0, 10):
        if(max_x < x[i]):
            max_x = x[i]
            idx = i
            
    return idx

# Define the normalize function.

In [5]:
def normalize(data):

    data_normalized = (data - min(data)) / (max(data) - min(data))

    return(data_normalized)

# Make a matrix each column of which represents an images in a vector form.

In [6]:
list_image_train    = np.empty((size_row * size_col, num_train), dtype=float)
list_label_train    = np.empty(num_train, dtype=int)

list_image_test     = np.empty((size_row * size_col, num_test), dtype=float)
list_label_test     = np.empty(num_test, dtype=int)

# Define a function to get each theta in various parameters.

In [7]:
def get_theta(select,p, n, r):
    count = 0
    for line in data_train:

        line_data   = line.split(',')
        label       = line_data[0]
        im_vector   = np.asfarray(line_data[1:])
        im_vector   = normalize(im_vector)

        if(label == str(select)):
            list_label_train[count] = 1
        else :
                list_label_train[count] = -1

        list_image_train[:, count]  = im_vector 

        count += 1
    
    A = np.empty((num_train,p[n]), dtype=float)

    for i in range(0, num_train):
        A[i] = f(list_image_train[:,i], p[n], r)
        
    theta = np.dot(np.dot(np.linalg.inv(np.dot(A.T,A)),A.T),list_label_train[:])
    
    return theta

# Get the test data.

In [8]:
count = 0

for line in data_test:

    line_data   = line.split(',')
    label       = line_data[0]
    im_vector   = np.asfarray(line_data[1:])
    im_vector   = normalize(im_vector)

    for j in range(0, 10):
        if(label == str(j)):
            list_label_test[count] = j
        
    list_image_test[:, count]   = im_vector    

    count += 1

# Calculate the theta values and visualize them with various parameters.

In [9]:
max_f1 = 0;
    
for n in range(0, 4):
    
    c_m = np.zeros((10, 10), dtype=int)
    
    r = np.random.normal(0, 1, (p[n],size_row*size_col))
    
    theta = np.empty((10,p[n]),dtype=float)
    
    for i in range(0, 10):
        theta[i] = get_theta(i, p, n, r)
        

    X = np.empty((num_test, p[n]), dtype=float)
    
    for i in range(0, num_test):
        X[i] = f(list_image_test[:,i], p[n],r)
        
    tp = 0
    fp = 0
    fn = 0
    
    for i in range(0, num_test) :
        
        guesses = np.zeros(10, dtype=float)
        
        for j in range(0, 10):
            guesses[j] = np.dot(X[i], theta[j].T)
            
        guess = argmax(guesses)
            
        for l in range(0, 10):
            for k in range(0, 10):
                if(guess == l and list_label_test[i] == k):
                    c_m[l][k] = c_m[l][k] + 1
                    if(l == k):
                        tp = tp + 1
                    elif(l > k):
                        fp = fp + 1
                    elif(l < k):
                        fn = fn + 1
    print("Confusion Matrix when p = ", p[n])                    
    print(c_m)
    
    precision = (tp / (tp + fp))
    recall = (tp / (tp + fn))
    
    f1 = 2 * ((precision * recall) / (precision + recall))
    
    print("F1 Score is ", f1 , " when p = ", p[n])
    
    if(f1 > max_f1):
        max_f1 = f1 

Confusion Matrix when p =  5
[[704  32 213 210  55 423 318  15 141  20]
 [ 23 580  57  68 145  25  31  81  62  66]
 [  5  15  27   5  17   5  20  14  10   8]
 [ 41   2  43 253  76  76  63  43  44  59]
 [  4   8  18   9  24   6  14  11   6  22]
 [  0   0   1  14   1  11   1   0   0   0]
 [ 59   9 105  41  44  73 207  19  37  21]
 [  5  77  76  88 137  57  20 397  45 143]
 [ 87 250 342 188 125 169 197 170 582  81]
 [ 52 162 150 134 358  47  87 278  47 589]]
F1 Score is  0.5045610886795274  when p =  5
Confusion Matrix when p =  25
[[ 805    1   29   60   25   69   96   40   18   52]
 [  11 1041   72   57   30   85   36   37   73   30]
 [  12   29  725   70   32   19   42   19   26   10]
 [  24   19   33  657    8  158    0   20   79   21]
 [   5   15   37    3  660   28   33   56   42  184]
 [   8    3    6   71   21  337    7   11   35   33]
 [  61    7   45   23   48   47  724    6   46   42]
 [   9    1   25   31   17   48    4  745   25  113]
 [  38   18   54   33   85   71   10   15

# Calculate the best F1 score.

In [10]:
print("The best F1 Score is ", max_f1)

The best F1 Score is  0.9207274297123738
